# Examples
This chapter introduces the primary use of MoHa calculation, and you can find those examples in the `moharoot/data/examples` directory. The accompanying user guide will present additional details on the various modules.


To begin a MoHa calculation, first build the Hamiltonian of a system. In terms of second quantization operators, a time-independent non-relativistic Hamiltonian gives:
\begin{equation}
H = - \sum_{ij} t_{ij}\hat{c}^{\dagger}_{i}\hat{c}_{j} + \frac{1}{2} \sum_{ijkl}
V_{ijkl}\hat{c}^{\dagger}_{i}\hat{c}^{\dagger}_{k}\hat{c}_{l}\hat{c}_{j}
\end{equation}

The construction of the molecular Hamiltonian is set in three steps.

- Construct a molecular geometry.
- Generate a basis set for the molecular.
- Generate all kinds of operators with molecule and basis set to define a Hamiltonian.


## Hartree-Fock
The Hartree-Fock method is an uncorrelated mean-field theory that offers a qualitative description of chemical systems. Although Hartree-Fock theory is only qualitatively correct, it forms the basis for more accurate models and becomes the cornerstone of ab initio quantum chemistry.


### Plain Solver
A basic Hartree-Fock calculation with a plain solver can be performed by:

In [1]:
from moha import *

log.output('silent')

geo = [[8,   0.000000000000,  -0.143225816552,   0.000000000000],
    ['h',   1.638036840407,   1.136548822547,  -0.000000000000],
    ["H",  -1.638036840407,   1.136548822547,  -0.000000000000]]

mol = Molecule.build(geo,pg=False)
orb = BasisSet.build(mol,'sto-3g.nwchem')

ham = Hamiltonian.build(mol,orb)
sym = Symmetry(n=10,ms2=2,ipg=0)

hf_solver = HFSolver(ham,sym)
hf_energy,hf_wfn = hf_solver.uhf()

In [2]:
print(hf_energy+mol.e_nuc)

-74.68932024828379


### DIIS Solver
The convergence of the plain Hartree-Fock solver is relatively slow. One of the most effective methods for speeding up such iterative sequences is the "Direct Inversion in the Iterative Subspace(DIIS)" approach.


In [5]:
from moha import *

log.output('silent')

geo = [[8,   0.000000000000,  -0.143225816552,   0.000000000000],
    ['h',   1.638036840407,   1.136548822547,  -0.000000000000],
    ["H",  -1.638036840407,   1.136548822547,  -0.000000000000]]

mol = Molecule.build(geo,pg=False)
orb = BasisSet.build(mol,'sto-3g.nwchem')

ham = Hamiltonian.build(mol,orb)

hf_solver = HFSolver(ham,Symmetry(10,0,0))
hf_energy,hf_wfn = hf_solver.rhf(acceleration=True)

In [4]:
print(hf_energy+mol.e_nuc)

-74.94207992723523


## Post-Hartree-Fock
Quantum chemistry is mature enough to have canonical approaches followed by the Hartree-Fock method. Their behavior, successes, and most importantly, their failures are widely known and accepted. With a couple of exceptions which we will not mention, they are:

- Configuration Interaction(CI)
- Coupled-Cluster(CC)
- Perturbation Theory(PT)


### Configuration Interaction
Configuration interaction approximates wave function by a linear expansion of N-electron basis functions made of a given one-electron basis. With CI wave function and CI basis set, the Schrödinger equation becomes a matrix-eigenvalue equation.


#### Full CI
The full configuration interaction(FCI) method assumes that all electrons are correlated among all orbitals in a given system. Hence it provides numerically exact solutions (within the infinitely flexible complete basis set) to the electronic time-independent, non-relativistic Schrödinger equation.

In [6]:
from moha import *

log.output('silent')

geo = [[8,   0.000000000000,  -0.143225816552,   0.000000000000],
    ['h',   1.638036840407,   1.136548822547,  -0.000000000000],
    ["H",  -1.638036840407,   1.136548822547,  -0.000000000000]]

mol = Molecule.build(geo,pg=False)
orb = BasisSet.build(mol,'sto-3g.nwchem')

ham = Hamiltonian.build(mol,orb)
sym = Symmetry(n=10,ms2=2,ipg=0)

hf_solver = HFSolver(ham,sym)
hf_results,hf_wfn = hf_solver.hf()

#### CISD
FCI method is exact in a given atomic orbital basis but prohibitively expensive. Therefore, to balance accuracy and computational time, we truncate the CI space by excitation level relative to the reference state. The most common truncation of the CI space expansion is CI singles and doubles (CISD).


In [7]:
from moha import *

log.output('silent')

geo = [[8,   0.000000000000,  -0.143225816552,   0.000000000000],
    ['h',   1.638036840407,   1.136548822547,  -0.000000000000],
    ["H",  -1.638036840407,   1.136548822547,  -0.000000000000]]

mol = Molecule.build(geo,pg=False)
orb = BasisSet.build(mol,'sto-3g.nwchem')

ham = Hamiltonian.build(mol,orb)
sym = Symmetry(n=10,ms2=2,ipg=0)

hf_solver = HFSolver(ham,sym)
hf_results,hf_wfn = hf_solver.hf()

### Coupled-Cluster
Coupled cluster methods are among the most accurate electronic structure methods available today. Its wave function ansatz gives in exponential form $e^{\hat{T}} \vert\Phi_0\rangle$

Where the cluster operator $\hat{T}$ is the sum of $\{\hat{T}_{i}\}$ generates all possible determinants having the i-th excitations from the reference $\vert\Phi_0\rangle$.

#### CCSD

Truncating the cluster operator $\hat{T}$ at doubles leads to the coupled cluster singles and doubles method(CCSD). By keeping only $\hat{T}_1$ and $\hat{T}_2$, we reach the balance between accuracy and computational time.


In [8]:
from moha import *

log.output('silent')

geo = [[8,   0.000000000000,  -0.143225816552,   0.000000000000],
    ['h',   1.638036840407,   1.136548822547,  -0.000000000000],
    ["H",  -1.638036840407,   1.136548822547,  -0.000000000000]]

mol = Molecule.build(geo,pg=False)
orb = BasisSet.build(mol,'sto-3g.nwchem')

ham = Hamiltonian.build(mol,orb)
sym = Symmetry(n=10,ms2=2,ipg=0)

hf_solver = HFSolver(ham,sym)
hf_results,hf_wfn = hf_solver.hf()

#### CCSD(T)

Based on CCSD, the addition of perturbative triple correction gives the CCSD(T) method the “gold standard” in computational chemistry. It is now one of the most accurate methods applicable to reasonably large molecules.

In [ ]:
from moha import *

log.output('silent')

geo = [[8,   0.000000000000,  -0.143225816552,   0.000000000000],
    ['h',   1.638036840407,   1.136548822547,  -0.000000000000],
    ["H",  -1.638036840407,   1.136548822547,  -0.000000000000]]

mol = Molecule.build(geo,pg=False)
orb = BasisSet.build(mol,'sto-3g.nwchem')

ham = Hamiltonian.build(mol,orb)
sym = Symmetry(n=10,ms2=2,ipg=0)

hf_solver = HFSolver(ham,sym)
hf_results,hf_wfn = hf_solver.hf()

### Perturbation Theory
Perturbation theory is a collection of versatile methods used in many branches of science. It divides the system into a model part $\hat{H}_0$ which is a known approximation to the real system $\hat{H}$ and a perturbation part $\lambda\hat{V}$, where $\lambda$ is a parameter that is small enough to guarantee convergence.

Møller-Plesset perturbation theory is a particular case of perturbation theory, where we take the Fock operator $\hat{F}$ as the model operator.


#### MP2

Depending on the choice of Møller-Plesset perturbation operator $\hat{V}$, the first order correction energy $E_{MP1}$ might be zero or none zero, but $E_{MP0} + E_{MP1}$ is always equal to the Hartree-Fock energy $E_{HF}$. Hence, the first meaningful correction in Møller-Plesset perturbation theory is second-order energy.


In [16]:
from moha import *

log.output('silent')

geo = [[8,   0.000000000000,  -0.143225816552,   0.000000000000],
    ['h',   1.638036840407,   1.136548822547,  -0.000000000000],
    ["H",  -1.638036840407,   1.136548822547,  -0.000000000000]]

mol = Molecule.build(geo,pg=False)
orb = BasisSet.build(mol,'sto-3g.nwchem')

ham = Hamiltonian.build(mol,orb)
sym = Symmetry(n=10,ms2=2,ipg=0)

hf_solver = HFSolver(ham,sym)
hf_results,hf_wfn = hf_solver.hf()

mp_solver = MPSolver(ham,hf_wfn)
mp_results = mp_solver.mp2()

In [17]:
print(mp_results)

-74.71555403713197


## Property
This section contains information about atomic and molecular properties implemented in MoHa.

### Population Analyasis

Population analysis is the study of charge distribution in molecules. It requires the overlap integrals and the electron density and information about the number of basis functions centered on each atom.

In [ ]:
from moha import *

log.output('silent')

geo = [[8,   0.000000000000,  -0.143225816552,   0.000000000000],
    ['h',   1.638036840407,   1.136548822547,  -0.000000000000],
    ["H",  -1.638036840407,   1.136548822547,  -0.000000000000]]

mol = Molecule.build(geo,pg=False)
orb = BasisSet.build(mol,'sto-3g.nwchem')

ham = Hamiltonian.build(mol,orb)
sym = Symmetry(n=10,ms2=2,ipg=0)

hf_solver = HFSolver(ham,sym)
hf_results,hf_wfn = hf_solver.hf()

### Multipole Moment

Multipole moments are the coefficients of a series expansion of a potential due to continuous or discrete sources. A multipole moment usually involves powers (or inverse powers) of the distance to the origin and some angular dependence.


In [ ]:
from moha import *

log.output('silent')

geo = [[8,   0.000000000000,  -0.143225816552,   0.000000000000],
    ['h',   1.638036840407,   1.136548822547,  -0.000000000000],
    ["H",  -1.638036840407,   1.136548822547,  -0.000000000000]]

mol = Molecule.build(geo,pg=False)
orb = BasisSet.build(mol,'sto-3g.nwchem')

ham = Hamiltonian.build(mol,orb)
sym = Symmetry(n=10,ms2=2,ipg=0)

hf_solver = HFSolver(ham,sym)
hf_results,hf_wfn = hf_solver.hf()